# Improving document retrieval

## load and format test cases

In [1]:
import re

In [2]:
with open('../data/test_cases.txt') as f:
    test_cases_raw = f.readlines()

In [3]:
test_cases_raw = "".join(test_cases_raw[0:-1:2]).split('问：')[1:]
test_cases_raw[:3]

['什么是我国第一部编年史著作？\n答：《左传》。\n资料：附：《左传》是我国第一部编年史著作。\n',
 '什么是我国第一部编年国别史？\n答：《国语》。\n资料：附：《国语》是我国第一部编年国别史。\n',
 '“寡人之于国也”下一句是什么？来自哪里？\n答：“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。\n资料：梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”\n资料：《寡人之于国也》（孟子）\n']

In [4]:
test_cases_raw = [re.split(r'\n答：|\n资料：', x) for x in test_cases_raw]
test_cases_raw[:3]

[['什么是我国第一部编年史著作？', '《左传》。', '附：《左传》是我国第一部编年史著作。\n'],
 ['什么是我国第一部编年国别史？', '《国语》。', '附：《国语》是我国第一部编年国别史。\n'],
 ['“寡人之于国也”下一句是什么？来自哪里？',
  '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  '梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
  '《寡人之于国也》（孟子）\n']]

In [5]:
test_cases = [{'q': x[0], 'a': x[1], 'docs': x[2:]} for x in test_cases_raw]
test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [6]:
test_cases[-3:-1]

[{'q': '“我相信某一天，在某个人的生命中，他头顶天空中所有云朵的形状将会片刻地与他相似。”出自哪里？',
  'a': '原文来自“我们生活的故事 【美】马克·斯特兰德”中《纪念碑》的节选。',
  'docs': ['《纪念碑》（节选）\n']},
 {'q': '有一个诗人曾经写过类似“有人把国家带进沟里，但这些人说统治是一门艺术”的话。请找到出处。',
  'a': '在“致后代 【德】贝托尔特·布莱希特”的“那些把肉从桌上拿走的人【1937】”中，作者写道：\\n那些把肉从桌上拿走的人\\n教导人们满足。\\n那些获得进贡的人\\n要求人们牺牲。\\n那些吃饱喝够的人向饥饿者\\n描绘将来的美好时代。\\n那些把国家带到深渊里的人\\n说统治太难，普通人\\n不能胜任。',
  'docs': ['那些把肉从桌上拿走的人\\n教导人们满足。\\n那些获得进贡的人\\n要求人们牺牲。\\n那些吃饱喝够的人向饥饿者\\n描绘将来的美好时代。\\n那些把国家带到深渊里的人\\n说统治太难，普通人\\n不能胜任。',
   '那些把肉从桌上拿走的人【1937】',
   '致后代 【德】贝托尔特·布莱希特\n']}]

## load Notion DB

In [1]:
from langchain_community.document_loaders import NotionDBLoader
import tomllib

In [2]:
with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.notion_databases.toml', 'rb') as f:
    _DATABASES_NOTION = tomllib.load(f)

In [3]:
_DATABASES_NOTION

{'写作': '8b81c07089344d8d95ddba22bafb8c12',
 '笔记（非文学）': '7443174d151342458fb7e47acdbb0c64',
 '读书笔记（文学）': '808e50735cce4c66a151f3e8b79c8d9d',
 '格言小集': '04d75f24b9544f3f8b220d5e7d87b7c3'}

In [4]:
# # for testing, only use 2 with test cases
# del _DATABASES_NOTION['写作']
# del _DATABASES_NOTION['笔记（非文学）']

In [5]:
_DATABASES_NOTION

{'写作': '8b81c07089344d8d95ddba22bafb8c12',
 '笔记（非文学）': '7443174d151342458fb7e47acdbb0c64',
 '读书笔记（文学）': '808e50735cce4c66a151f3e8b79c8d9d',
 '格言小集': '04d75f24b9544f3f8b220d5e7d87b7c3'}

In [6]:
def load_notion_dbs(dbs, id):
    loader = NotionDBLoader(
        integration_token=_TOKENS['notion'],
        database_id=dbs[id],
        request_timeout_sec=300,  # optional, defaults to 10
    )
    data = loader.load()
    return data

In [15]:
for k in _DATABASES_NOTION:
    _ = load_notion_dbs(_DATABASES_NOTION, k)

HTTPError: 400 Client Error: Bad Request for url: https://api.notion.com/v1/databases/8b81c07089344d8d95ddba22bafb8c12/query

In [7]:
%%time
from concurrent.futures import ThreadPoolExecutor

e = ThreadPoolExecutor()
results = list(e.map(lambda x: load_notion_dbs(_DATABASES_NOTION, x), _DATABASES_NOTION.keys()))

CPU times: user 8.77 s, sys: 806 ms, total: 9.57 s
Wall time: 1min 28s


In [8]:
len(results)

4

In [13]:
results[2][1]

Document(page_content='中国哲学的精神\n中国文化的精神基础是\n伦理\n（特别是儒家伦理）而不是宗教\n中国人不以宗教观念和宗教活动为生活中最重要、最迷人的部分。这和其他主要文化（以寺院、僧侣为主导）有根本的重要的不同\n中国人在哲学里满足了他们对超乎现世的追求，也在哲学里表达、欣赏和体验超道德价值\n人们习惯于说中国有三教：儒教、道教、佛教。但其实儒教比起宗教更是一种伦理、哲学或者文化；作为一个哲学学派的道家（顺其自然）和以研究方术避免死亡的道教（反乎自然）也有区别；同理，作为哲学的佛学比宗教的佛教在社会上的影响大得多\n为道 ≠ 为学。在中国哲学的传统中，哲学并不在于增加积极的、实用的知识，而在于提高心灵的境界，获得高于道德价值的价值\n中国的哲学既是出世的也是入世的\n“不离日用常行内，直到先天未画前”：中国哲学寻求解决的问题，是把入世和出世这两个对立的命题统一成一个合命题。它是最理想主义的，也是最现实主义的\n“内圣外王”：中国哲学中，如果一个人不仅在理论而且在行动上能够完成这种统一，他就是圣人，也是最适宜为王之人。这和柏拉图“哲学家-王”的理论很有相似之处\n中国的哲学和政治思想无法分开：儒家认为，处理日常的人伦世务，不是圣人的分外之事。处理世务，正是圣人人格完全发展的实质所在。他不仅作为社会的公民，而且也作为孟子所谓“天民”，来执行这个任务。正如柏拉图的《理想国》既代表了他的整个哲学，同时也是他的政治思想\n中国的哲学家以身载道：这个哲学家遵守他的哲学信念而生活，他的生活是他的哲学的组成部分。中国哲学家都是不同程度的苏格拉底，对于他们，哲学从来就不是为人类认识摆设的观念模式，而是内在与他的行动的箴言体系；在极端情况下，他的哲学简直可以说是他的传记\n中国哲学的阐述多依靠不明晰的比喻论证，而没有正式哲学著作\n中国哲学家看来，研究哲学不是一种职业，因此中国没有职业哲学家\n研究中国的哲学，只有看这些人的语录或者写给学生、朋友的信。这些片段来自于他一生的各个时期，语录也不只是一人所记。所以它们不相联系，有时甚至互相矛盾\n中国哲学家惯用名言隽语、比喻例证来表达自己的思想。这种表达明晰不足而暗示有余。明晰和暗示是不可兼得的，正如一种表达越是散文化就越缺乏诗意；而富于暗示的倾向性是中国一切艺术的理想（“言有尽而意无穷”）\n郭象是《庄子》

In [9]:
loader = NotionDBLoader(
    integration_token=_TOKENS['notion'],
    database_id=_DATABASES_NOTION['格言小集'],
)
_ = loader.load().copy()

TypeError: NotionDBLoader.__init__() got an unexpected keyword argument 'verbose'

In [19]:
type(_)

list

In [10]:
loader = NotionDBLoader(
    integration_token=_TOKENS['notion'],
    database_id=_DATABASES_NOTION['读书笔记（文学）'],
)
_ = loader.load().copy()

In [9]:
_[:2]

[Document(page_content='人生四不捡：塔吊下边冰红茶，铁轨上边牛肉干，过山车下八宝粥，课桌后边葡萄干', metadata={'name': '笑死', 'id': '2303056b-72f1-4f9a-b377-cf6b12a0d424'}),
 Document(page_content='\n名为太阳的溶剂\n\n楼群之中，乌鸦堕落为一种失去预言性的可厌生物\n\n对大多数人来说，他们所抱定的观念只是为他们生存方式做辩护的使用说明书而已。\n\n颤抖的彩虹其实是光的边角余料。\n\n以前的建筑是不透明的甲壳类，现在的建筑是透光的腔肠类。\n\n射电望远镜就是宇宙射线的向日葵。\n\n电线像静脉一样生长着。\n\n绝望的尽头并非对绝望感到绝望，而是对绝望感到厌烦。相对地，希望的尽头也只是对希望的疲倦而已。与一般的见解不同，绝望和希望是生活的激发态。\n', metadata={'name': '沃滋集朔德', 'id': '340f4781-1042-4534-b286-b758b0ca09a4'})]

## Testing ideas


* langchain.text_splitter --> MarkdownHeaderTextSplitter
    * only one choice of splitter because we are dealing with markdown files
    * parameters to game:
        * strip_headers: True vs. False (hunch should keep True)
        * chunk_size = 128, 256, 512
        * chunk_overlap  = 50, 100, 200
* langchain_community.embeddings
    * from langchain_openai import OpenAIEmbeddings
    * model = SentenceTransformer('intfloat/e5-large-v2') This model has 24 layers and the embedding size is 1024.
    * sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
    * from langchain_community.embeddings import OllamaEmbeddings
    * from langchain_community.embeddings import LlamaCppEmbeddings
    * from langchain_community.embeddings import GPT4AllEmbeddings
    * from langchain_google_vertexai import VertexAIEmbeddings
    * from langchain_community.embeddings import CohereEmbeddings
    * from langchain_community.embeddings import QianfanEmbeddingsEndpoint
* langchain_community.vectorstores
* retrieval methods: cos/dot; llm-aided; MMR (Maximum marginal relevance)

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter